### Query 2 with SQL API

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
import time

# Start Spark session
spark = SparkSession.builder.appName("Query 2 - SQL API").getOrCreate()

# Define the schema for the crime data
crimeSchema = StructType([
    StructField("DR_NO", StringType(), True),
    StructField("Date Rptd", StringType(), True),
    StructField("DATE OCC", StringType(), True),
    StructField("TIME OCC", StringType(), True),
    StructField("AREA", StringType(), True),
    StructField("AREA NAME", StringType(), True),
    StructField("Rpt Dist No", StringType(), True),
    StructField("Part 1-2", StringType(), True),
    StructField("Crm Cd", StringType(), True),
    StructField("Crm Cd Desc", StringType(), True),
    StructField("Mocodes", StringType(), True),
    StructField("Vict Age", StringType(), True),
    StructField("Vict Sex", StringType(), True),
    StructField("Vict Descent", StringType(), True),
    StructField("Premis Cd", StringType(), True),
    StructField("Premis Desc", StringType(), True),
    StructField("Weapon Used Cd", StringType(), True),
    StructField("Weapon Desc", StringType(), True),
    StructField("Status", StringType(), True),
    StructField("Status Desc", StringType(), True),
    StructField("Crm Cd 1", StringType(), True),
    StructField("Crm Cd 2", StringType(), True),
    StructField("Crm Cd 3", StringType(), True),
    StructField("Crm Cd 4", StringType(), True),
    StructField("LOCATION", StringType(), True),
    StructField("Cross Street", StringType(), True),
    StructField("LAT", StringType(), True),
    StructField("LON", StringType(), True)
])

# File paths for crime data
file1 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
file2 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"

start_time = time.time()

# Read the crime data
df1 = spark.read.csv(file1, header=True, schema=crimeSchema)
df2 = spark.read.csv(file2, header=True, schema=crimeSchema)

# Union the datasets
df = df1.union(df2)

# Register table for SQL operations
df.createOrReplaceTempView("crime_data")

# Convert `DATE OCC` to timestamp and extract the year
spark.sql("""
    SELECT *, 
        YEAR(TO_TIMESTAMP(`DATE OCC`, 'MM/dd/yyyy hh:mm:ss a')) AS Year
    FROM crime_data
""").createOrReplaceTempView("crime_with_year")

# Calculate total and closed cases, and closed percentage directly
spark.sql("""
    SELECT 
        Year,
        `AREA NAME`, 
        COUNT(*) AS Total_Cases,
        (SUM(CASE 
            WHEN `Status Desc` NOT IN ('UNK', 'Invest Cont') THEN 1 
            ELSE 0 
        END) / COUNT(*)) * 100 AS Closed_Percentage
    FROM crime_with_year
    GROUP BY Year, `AREA NAME`
""").createOrReplaceTempView("cases_with_percentage")

# Rank departments by closed percentage
spark.sql("""
    SELECT 
        Year, 
        `AREA NAME`, 
        Total_Cases, 
        Closed_Percentage, 
        RANK() OVER (PARTITION BY Year ORDER BY Closed_Percentage DESC) AS Rank
    FROM cases_with_percentage
""").createOrReplaceTempView("ranked_departments")

# Filter top 3 departments per year and sort by year and rank
top_departments = spark.sql("""
    SELECT 
        Year, 
        `AREA NAME`, 
        Closed_Percentage, 
        Rank 
    FROM ranked_departments
    WHERE Rank <= 3
    ORDER BY Year, Rank
""")

# Show results
top_departments.show(60, truncate=False)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")
